# Portfolio Performance vs. Benchmark

### Step 1: Import Libraries

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Visualization libraries
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image, display

# System libraries
import os
import sys
import glob
import logging
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # Font warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*inplace.*")  # Pandas warnings
logging.getLogger('matplotlib.font_manager').disabled = True

In [2]:
from py.quantstats_fix import *
from py.utils import load_and_filter_data, export_to_excel
qs.extend_pandas()

               QuantStats Compatibility Tool                

Part 1: Directly patching QuantStats package files
------------------------------------------------------------
Found QuantStats utils file at: /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/__init__.py
Successfully fixed indentation in QuantStats __init__.py file
✓ QuantStats utils file patched successfully

Part 2: Fixing resampling issues
------------------------------------------------------------
Found 1 potential QuantStats installation(s)
Checking /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ Found 'plot_timeseries' function in /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ No 'sum(axis=0)' calls found - may already be fixed
Examining /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py...
✓ Found 'plot_timeserie

### Step 2: Define Parameters 

#### Dates

In [3]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')  # Report date
start_date = end_date - timedelta(days=10*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2015-06-09 to 2025-06-06
Time span: 3650 days (10.00 years)


#### Report File

In [4]:
# Get the most recent portfolio file
portfolio_files = glob.glob('portfolios/portfolio-*.xlsx')
if portfolio_files:
    # Sort files by modification time (most recent first)
    output_file = max(portfolio_files, key=os.path.getmtime)
    print(f"Using most recent portfolio file: {output_file}")
else:
    # Fallback to current date if no files found
    output_file = f'portfolios/portfolio-{datetime.date(end_date)}.xlsx'
    print(f"No portfolio files found. Using: {output_file}")

Using most recent portfolio file: portfolios/portfolio-2025-06-06.xlsx


#### Benchmark

In [5]:
# benchmark = 'ADME'
benchmark = pd.read_excel(output_file, sheet_name="benchmark_long_term")['Benchmark'].values[0]
print(benchmark)

YYY


#### Risk-free rate (T-bill, %)

In [6]:
# Load and process data
risk_free_df = pd.read_excel(output_file, sheet_name="daily_quotes", index_col=0)['^IRX']
# risk_free_rate  = 0.0433                              # 3‑month T‑Bill
risk_free_rate = risk_free_df.iloc[-1] / 100 

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")

Risk-Free Rate: 0.0424 -- 13 WEEK TREASURY BILL (^IRX)


### Step 3: Read Portfolio Data (Excel)

In [7]:
portfolio_df = pd.read_excel(output_file, sheet_name="long_term")

# Convert percentage strings to float values
portfolio_df['Weight'] = portfolio_df['Weight'].replace('%', '', regex=True).astype(float)
portfolio_tickers = portfolio_df["Ticker"].tolist()

print(portfolio_tickers)
display(portfolio_df)

['MMC', 'AMAT', 'TMUS', 'MRK', 'APO']


,Ticker,Date,Name,Sector,Industry,Country,Website,Market Cap,Enterprise Value,Float Shares,...,52W Low,50 Day Avg,200 Day Avg,Short Ratio,Short % of Float,Strategy,Weight,Expected Return,Standard Deviation,Sharpe Ratio
0,MMC,2025-06-02,"Marsh & McLennan Companies, Inc.",Financial Services,Insurance Brokers,United States,https://www.marshmclennan.com,115130826752,136148770816,491821141,...,205.17,229.54,225.30,2.62,0.01,Markowitz,0.318087,0.140449,0.212929,0.659604
1,AMAT,2025-06-02,"Applied Materials, Inc.",Technology,Semiconductor Equipment & Materials,United States,https://www.appliedmaterials.com,125791559680,125714595840,799392549,...,123.74,151.52,173.39,2.58,0.02,Markowitz,0.228911,0.181314,0.412511,0.439538
2,TMUS,2025-06-02,"T-Mobile US, Inc.",Communication Services,Telecom Services,United States,https://www.t-mobile.com,275005997056,384693796864,452736028,...,171.18,251.20,232.97,3.11,0.03,Markowitz,0.223672,0.130260,0.275833,0.472243
3,MRK,2025-06-02,"Merck & Co., Inc.",Healthcare,Drug Manufacturers - General,United States,https://www.merck.com,192947535872,218628636672,2507088934,...,73.31,81.27,97.40,2.49,0.02,Markowitz,0.145330,0.095043,0.220747,0.430553
4,APO,2025-06-02,"Apollo Global Management, Inc.",Financial Services,Asset Management,United States,https://www.apollo.com/institutional/homepage,74688552960,94705582080,446479870,...,95.11,133.28,143.86,7.75,0.05,Markowitz,0.084000,0.135080,0.379183,0.356240


### Step 4: Download Returns

In [8]:
# stock_returns = qs.utils.download_returns(ticker=portfolio_tickers, period="5y").dropna()
stock_quotes = load_and_filter_data('data/daily_stock_quotes.csv', portfolio_tickers, start_date, end_date)
stock_returns = np.log(stock_quotes / stock_quotes.shift(1)).dropna()

benchmark_quotes = load_and_filter_data('data/daily_benchmark_quotes.csv', benchmark, start_date, end_date)
benchmark_returns = np.log(benchmark_quotes / benchmark_quotes.shift(1)).dropna()

# Display summary statistics for all assets
print("\nSharpe Ratios for individual assets:")
sharpes = {col: qs.stats.sharpe(stock_returns[col]) for col in stock_returns.columns}
for ticker, sharpe in sharpes.items():
    print(f"{ticker}: {sharpe:.4f}")

display(stock_returns.head())

Found 5 of 5 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []

Sharpe Ratios for individual assets:
AMAT: 0.5366
APO: 0.6052
MMC: 0.7596
MRK: 0.2805
TMUS: 0.7018


,AMAT,APO,MMC,MRK,TMUS
Date,,,,,
2015-06-11,-0.013028,0.002215,0.008008,-0.004307,-0.014117
2015-06-12,-0.008014,-0.005917,-0.003796,-0.017905,0.022807
2015-06-15,0.004587,-0.005206,-0.005017,-0.013024,-0.001049
2015-06-16,0.005136,-0.000746,0.003414,0.010335,-0.002891
2015-06-17,-0.013754,0.001491,0.003203,0.003421,0.023414


### Step 6: Split Data into Training and Testing Sets

In [9]:
# Split data into 80% training and 20% testing based on date
total_rows = len(stock_quotes)
train_size = int(0.8 * total_rows)
training_set = stock_quotes.iloc[:train_size]
test_set = stock_quotes.iloc[train_size:]

print(f"Training: {len(training_set)} rows ({len(training_set)/total_rows:.1%}) | Testing: {len(test_set)} rows ({len(test_set)/total_rows:.1%})")
print(f"Training period: {training_set.index[0]} to {training_set.index[-1]}")
print(f"Testing period: {test_set.index[0]} to {test_set.index[-1]}")

Training: 2009 rows (80.0%) | Testing: 503 rows (20.0%)
Training period: 2015-06-10 00:00:00 to 2023-06-01 00:00:00
Testing period: 2023-06-02 00:00:00 to 2025-06-04 00:00:00


### Step 5: Plot Return Comparisons

In [10]:
# Calculate portfolio returns using weights from Excel file
portfolio_weights = portfolio_df.set_index('Ticker')['Weight'].to_dict()
weighted_returns = pd.DataFrame()

print("\nPortfolio Weights:")
for ticker in portfolio_tickers:
    weight = portfolio_weights.get(ticker, 0)
    print(f"{ticker}: {weight:.2%}")
    if ticker in stock_returns.columns:
        weighted_returns[ticker] = stock_returns[ticker] * weight

# Sum across all weighted returns to get the portfolio return
portfolio_return = weighted_returns.sum(axis=1)

# Create equal-weight portfolio for comparison
equal_weight = 1/len([t for t in portfolio_tickers if t in stock_returns.columns])
equal_weighted_returns = pd.DataFrame()

for ticker in portfolio_tickers:
    if ticker in stock_returns.columns:
        equal_weighted_returns[ticker] = stock_returns[ticker] * equal_weight
        
equal_weight_return = equal_weighted_returns.sum(axis=1)

print("\nPortfolio Performance Summary:")
print(f"Sharpe Ratio (Weighted Portfolio): {qs.stats.sharpe(portfolio_return):.4f}")
print(f"Sharpe Ratio (Equal-Weight): {qs.stats.sharpe(equal_weight_return):.4f}")

plt.figure(figsize=(12, 6))
(1 + portfolio_return).cumprod().plot(label='Weighted Portfolio')
(1 + equal_weight_return).cumprod().plot(label='Equal Weight')
(1 + benchmark_returns).cumprod().plot(label=benchmark)
plt.legend()
plt.title('Performance Comparison')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()


Portfolio Weights:
MMC: 31.81%
AMAT: 22.89%
TMUS: 22.37%
MRK: 14.53%
APO: 8.40%

Portfolio Performance Summary:
Sharpe Ratio (Weighted Portfolio): 0.8348
Sharpe Ratio (Equal-Weight): 0.8123


### Step 6: Generate Reports

In [11]:
import bt
import matplotlib.pyplot as plt

# Create weights dictionary using portfolio_df instead of optimal_portfolio_df
weights = {}
for _, row in portfolio_df.iterrows():
    ticker = row['Ticker']
    weight = row['Weight'] 
    weights[ticker] = weight

print("Portfolio weights:")
for ticker, weight in weights.items():
    print(f"{ticker}: {weight:.4f}")

# Pre-assign colors to tickers for consistency across all plots
ticker_color_map = {}
base_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', 
               '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#aec7e8', '#ffbb78']

for i, ticker in enumerate(portfolio_tickers):
    ticker_color_map[ticker] = base_colors[i % len(base_colors)]

print("\nTicker color assignments:")
for ticker, color in ticker_color_map.items():
    print(f"{ticker}: {color}")

# Create price data for bt (convert from quotes to prices)
# Combine stock quotes and benchmark quotes
all_quotes = test_set.copy()
all_quotes[benchmark] = benchmark_quotes[benchmark]

# Create bt strategies
# Strategy 1 - Portfolio with quarterly rebalancing
portfolio_strategy = bt.Strategy('Portfolio', 
                                [
                                    bt.algos.RunEveryNPeriods(66, offset=66),  # Trading every 66 days (approx. quarterly)                 
                                    bt.algos.SelectAll(),
                                    bt.algos.WeighSpecified(**weights),       
                                    bt.algos.Rebalance()
                                ])

# Strategy 2 - Benchmark (buy and hold)
benchmark_strategy = bt.Strategy(f'{benchmark}',
                                [
                                    bt.algos.RunEveryNPeriods(66, offset=66),
                                    bt.algos.SelectThese([benchmark]),
                                    bt.algos.WeighEqually(),
                                    bt.algos.Rebalance()
                                ])

# Create backtests
portfolio_test = bt.Backtest(portfolio_strategy, all_quotes[portfolio_tickers])
benchmark_test = bt.Backtest(benchmark_strategy, all_quotes[[benchmark]])

# Run backtests
result = bt.run(portfolio_test, benchmark_test)
result.set_riskfree_rate(risk_free_rate)

# Show performance metrics
result.display()
display(result.get_transactions())

# Plot weights over time using pre-assigned colors
fig, ax = plt.subplots(figsize=(10, 6))

# Get the weight data from the result - access it through the strategy result
portfolio_result = result['Portfolio Strategy']

ax.set_title('Portfolio Weights Over Time', fontsize=14, fontweight='bold')
ax.set_ylabel('Weight', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot cumulative returns comparison
result.plot(figsize=(12, 8),
           ylabel='Cumulative Returns',
           title='Portfolio vs Benchmark Performance',
           legend=True,
           grid=True)

Portfolio weights:
MMC: 0.3181
AMAT: 0.2289
TMUS: 0.2237
MRK: 0.1453
APO: 0.0840

Ticker color assignments:
MMC: #1f77b4
AMAT: #ff7f0e
TMUS: #2ca02c
MRK: #d62728
APO: #9467bd


100%|██████████| 2/2 [00:00<00:00, 16.77it/s]


Stat                 Portfolio    YYY
-------------------  -----------  ----------
Start                2023-06-01   2023-06-01
End                  2025-06-04   2025-06-04
Risk-free rate       4.24%        4.24%

Total Return         29.04%       21.14%
Daily Sharpe         0.64         0.56
Daily Sortino        1.04         0.81
CAGR                 13.53%       10.01%
Max Drawdown         -13.94%      -14.44%
Calmar Ratio         0.97         0.69

MTD                  0.86%        0.70%
3m                   -2.88%       0.35%
6m                   -6.06%       -1.79%
YTD                  1.51%        3.13%
1Y                   2.86%        6.77%
3Y (ann.)            13.53%       10.01%
5Y (ann.)            -            -
10Y (ann.)           -            -
Since Incep. (ann.)  13.53%       10.01%

Daily Sharpe         0.64         0.56
Daily Sortino        1.04         0.81
Daily Mean (ann.)    13.92%       10.20%
Daily Vol (ann.)     15.18%       10.86%
Daily Skew           0.17   

price  quantity
Date       Security                  
2023-09-07 AMAT      146.43    1563.0
           APO        84.87     989.0
           MMC       191.45    1661.0
           MRK       102.72    1414.0
           TMUS      134.25    1666.0
2023-12-11 AMAT      153.58       8.0
           APO        90.01      -5.0
           MMC       195.06      58.0
           MRK       100.00     118.0
           TMUS      156.02    -155.0
2024-03-18 AMAT      199.05    -202.0
           APO       108.38     -61.0
           MMC       203.89     139.0
           MRK       117.94     -65.0
           TMUS      158.91     165.0
2024-06-21 AMAT      233.87     -95.0
           APO       116.21      18.0
           MMC       212.02      95.0
           MRK       127.72      15.0
           TMUS      174.94     -11.0
2024-09-25 AMAT      196.39     246.0
           APO       123.39     -54.0
           MMC       222.23     -87.0
           MRK       112.84     197.0
           TMUS      201.90    -221.0
2024-12-30 AMAT      163.26     261.0
           APO       165.45    -242.0
           MMC       211.19      48.0
           MRK        97.53     214.0
           TMUS      219.92    -152.0
2025-04-07 AMAT      132.85     303.0
           APO       111.96     262.0
           MMC       223.74    -194.0
           MRK        81.18     272.0
           TMUS      246.67    -195.0

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


<Axes: title={'center': 'Portfolio vs Benchmark Performance'}, ylabel='Cumulative Returns'>

#### Portfolio (Weighted) vs Benchmark

In [12]:
# Filter portfolio_return and benchmark_returns to match test_set dates
test_start_date = test_set.index[0]
test_end_date = test_set.index[-1]

# Filter portfolio_return to test period
portfolio_return_test = portfolio_return.loc[test_start_date:test_end_date]
portfolio_return_test.name = "Weighted Portfolio"

# Filter benchmark_returns to test period
benchmark_returns_test = benchmark_returns.loc[test_start_date:test_end_date]

print(f"Original portfolio_return length: {len(portfolio_return)}")
print(f"Filtered portfolio_return length: {len(portfolio_return_test)}")
print(f"Test period: {test_start_date} to {test_end_date}")

# Generate Report using filtered data
qs.reports.html(
    portfolio_return_test,
    benchmark_returns_test,
    rf=risk_free_rate,
    figsize=(8, 5),
    output=f'portfolios/portfolio_vs_benchmark_long_term-{datetime.date(end_date)}.html',
    title=f'Portfolio vs {benchmark} (Benchmark) - Test Period',
    benchmark_title=f'{benchmark}',
    download_filename=f'portfolio_vs_{benchmark}.html'
)

qs.reports.full(
    portfolio_return_test, 
    benchmark_returns_test,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title=f'Portfolio vs {benchmark} - Test Period',
    benchmark_title=f'{benchmark}')

Original portfolio_return length: 2511
Filtered portfolio_return length: 503
Test period: 2023-06-02 00:00:00 to 2025-06-04 00:00:00


/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retai

Added download button and removed QuantStats attribution from portfolios/portfolio_vs_benchmark_long_term-2025-06-06.html


/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retai

                           YYY         Strategy
-------------------------  ----------  ----------
Start Period               2023-06-02  2023-06-02
End Period                 2025-06-04  2025-06-04
Risk-Free Rate             4.24%       4.24%
Time in Market             90.0%       100.0%

Cumulative Return          24.18%      30.11%
CAGR﹪                     0.03%       0.04%

Sharpe                     0.65        0.65
Prob. Sharpe Ratio         68.08%      62.99%
Smart Sharpe               0.6         0.6
Sortino                    0.87        0.92
Smart Sortino              0.81        0.85
Sortino/√2                 0.62        0.65
Smart Sortino/√2           0.57        0.6
Omega                      1.12        1.12

Max Drawdown               -14.74%     -15.66%
Longest DD Days            122         184
Volatility (ann.)          11.2%       15.88%
R^2                        0.38        0.38
Information Ratio          0.01        0.01
Calmar                     0.52        0.6

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2024-12-03,2025-04-08,2025-06-04,184,-15.658337,-14.391188
2,2024-07-17,2024-08-05,2024-10-11,87,-9.844154,-9.279887
3,2023-09-15,2023-10-27,2023-11-09,56,-6.535265,-5.899937
4,2024-10-15,2024-10-31,2024-11-29,46,-5.160199,-5.000851
5,2023-12-14,2023-12-20,2024-01-11,29,-3.583005,-2.526976


/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/home/renanmogo/mfin-alg

#### Portfolio (Weighted) vs Portolio (Equally-Weighted)

In [13]:
# # Ensure equal_weight_return and portfolio_return are properly aggregated if resampled
# equal_weight_return = equal_weight_return.resample('D').sum() if equal_weight_return.index.freq is None else equal_weight_return
# portfolio_return = portfolio_return.resample('D').sum() if portfolio_return.index.freq is None else portfolio_return

# # Set the name for the equal weight portfolio
# equal_weight_return.name = "Equal Weight Portfolio"

# # Generate Report
# qs.reports.html(
#     portfolio_return,
#     equal_weight_return,
#     rf=risk_free_rate,
#     figsize=(8, 5),
#     output=f'portfolio_vs_equal_weight-{datetime.date(end_date)}.html',
#     title='Portfolio (Weighted) vs Portolio (Equally-Weighted)',
#     benchmark_title="Equal Weight Portfolio",
#     download_filename="portfolio_vs_equal_weight.html" 
# )

# qs.reports.full(
#     portfolio_return, 
#     equal_weight_return,
#     rf=risk_free_rate, 
#     figsize=(8, 5), 
#     title='Portfolio vs Equal Weight',
#     benchmark_title="Equal Weight Portfolio",
#     ) 